<a href="https://colab.research.google.com/github/Tian1398/IMG542-Project/blob/main/542NeworkProject_email.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# import packages
from google.colab import files
import os
import mailbox
import email
import json
import pandas as pd

# Load Data

In [5]:
def upload_mbox_files():
    # Prompt the user to upload multiple files
    uploaded = files.upload()

    file_paths = {}
    # Get the file names and paths
    for filename in uploaded.keys():
        print(f'Uploaded file: {filename}')
        file_path = os.path.abspath(filename)
        print(f'File path: {file_path}')
        file_paths[filename] = file_path
    return file_paths

# Call the function to upload mbox files
file_paths = upload_mbox_files()

# Load the mbox files into mailbox.mbox objects and save them as variables
def load_mbox_files(file_paths):
    mbox_data = {}
    for filename, file_path in file_paths.items():
        mbox = mailbox.mbox(file_path)
        mbox_data[filename] = mbox
    return mbox_data

# Call the load_mbox_files function to save the mbox data in variables
mbox_data = load_mbox_files(file_paths)

# Now you can access each mbox file using its filename as the key
for filename, mbox in mbox_data.items():
    print(f"Messages in {filename}: {len(mbox)}")

# Example: Accessing a specific mbox file's data
inbox_mbox = mbox_data.get('Inbox.mbox')
sent_mbox = mbox_data.get('Sent.mbox')

Saving Inbox.mbox to Inbox.mbox
Saving Sent.mbox to Sent (1).mbox
Uploaded file: Inbox.mbox
File path: /content/Inbox.mbox
Uploaded file: Sent (1).mbox
File path: /content/Sent (1).mbox
Messages in Inbox.mbox: 3531
Messages in Sent (1).mbox: 149


In [6]:
# Create empty lists to store the extracted data
inbox_data = []
sent_data = []

In [11]:
# select headers needed to be extracted
headers_to_extract = ['Date', 'From', 'To', 'Subject']

In [12]:
# Iterate through each message in the mbox file
for message in inbox_mbox:
    try:
        msg = email.message_from_bytes(message.as_bytes())
        email_data = {header: msg.get(header, None) for header in headers_to_extract}
        inbox_data.append(email_data)
    except Exception as e:
        print(f"Error processing message: {e}")

# Create a DataFrame
inbox_df = pd.DataFrame(inbox_data, columns=headers_to_extract)

# clean data: remove null
inbox_df = inbox_df.dropna(subset=['To'])

In [17]:
# Iterate through each message in the Sent mbox file
for message in sent_mbox:
    try:
        msg = email.message_from_bytes(message.as_bytes())
        email_data = {header: msg.get(header, None) for header in headers_to_extract}
        sent_data.append(email_data)
    except Exception as e:
        print(f"Error processing message: {e}")

# Create a DataFrame
sent_df = pd.DataFrame(sent_data, columns=headers_to_extract)
sent_df = sent_df.dropna(subset=['To'])

In [18]:
# Combine datasets
email_df = pd.concat([inbox_df, sent_df], ignore_index=True)
email_df.shape

(7097, 4)

In [19]:
email_df.head()

,Date,From,To,Subject
0,"Thu, 23 May 2024 22:32:24 +0000",iSchool Career Services <notifications@instruc...,tiancz23@uw.edu,Your Weekly Job Round-Up!: iSchool Career Serv...
1,"Tue, 21 May 2024 19:13:33 +0000",Handshake <handshake@g.joinhandshake.com>,tiancz23@uw.edu,You're invited to a virtual event with Handshake
2,"Mon, 13 May 2024 21:30:23 +0000",MSIM Academic Advising <iSchoolStudentServices...,Chengzi Tian <tiancz23@uw.edu>,Reminder: Early Career Accelerated - Virtual A...
3,"Mon, 13 May 2024 11:58:28 +0000","""Indeed"" <donotreply@indeed.com>",<tiancz23@uw.edu>,Financial Analyst @ Atlantic Group
4,"Sun, 26 May 2024 07:00:50 +0000 (UTC)",Ford Careers <noreplyr@ford.com>,tiancz23@uw.edu,"Confirm your identity for job Analyst, Digital..."


# Data Cleaning

Expand the elements where one person send a message to multiple recipients

In [21]:
# define a function to expand data by "To" column
def split_and_expand(row):
    to_addresses = row['To'].split('>,')
    expanded_rows = []
    for address in to_addresses:
        cleaned_address = address.strip().strip('<').strip()
        expanded_rows.append({
            'Date': row['Date'],
            'From': row['From'],
            'To': cleaned_address,
            'Subject': row['Subject']})

    return expanded_rows

# seperate the orginal dataframe to single-recipient vs. multiple recipients
single_way = inbox_df[~inbox_df['To'].str.contains(">,")]
multi_way = email_df[email_df['To'].str.contains(">,")]

# call functino to expand data on the section with multiple recipients
expanded_rows = []
for _, row in multi_way.iterrows():
    expanded_rows.extend(split_and_expand(row))

expanded_multi_df = pd.DataFrame(expanded_rows)

# merge the adjusted data again
expand_email_df = pd.concat([single_way, expanded_multi_df], ignore_index=True)
# check shape
expand_email_df.shape

(11010, 4)

convert my personal email address to "me"

In [23]:
def replace_email(value):
    if 'tiancz23' in value:
        return 'me'
    return value

# Apply the function to the 'From' and 'To' columns
expand_email_df['From'] = expand_email_df['From'].apply(replace_email)
expand_email_df['To'] = expand_email_df['To'].apply(replace_email)

In [25]:
# remove all quote punctuations
expand_email_df['From'] = expand_email_df['From'].str.replace(r'["\']', '', regex=True)
expand_email_df['To'] = expand_email_df['To'].str.replace(r'["\']', '', regex=True)

Clean sender and recipient names, remove email address info

In [27]:
# Function to keep only the part before the very first '@' if it exists
def keep_before_at(value):
    if '@' in value:
        return value.split('@')[0]
    return value

# Apply the function to the 'From' and 'To' columns
expand_email_df['From'] = expand_email_df['From'].apply(keep_before_at)
expand_email_df['To'] = expand_email_df['To'].apply(keep_before_at)

In [48]:
# clean wired spacing
expand_email_df['From'] = expand_email_df['From'].str.replace(r'(\n\\|\n|\t)', ' ', regex=True)
expand_email_df['To'] = expand_email_df['To'].str.replace(r'(\n\\|\n|\t)', ' ', regex=True)

In [49]:
# Function to keep only the part before the very first '<' if it exists
def keep_before_lt(value):
    if ' <' in value:
        return value.split('<')[0]
    return value

# Apply the function to the 'From' and 'To' columns
expand_email_df['From'] = expand_email_df['From'].apply(keep_before_lt)
expand_email_df['To'] = expand_email_df['To'].apply(keep_before_lt)

In [51]:
# trim cells
expand_email_df['From'] = expand_email_df['From'].str.strip()
expand_email_df['To'] = expand_email_df['To'].str.strip()

In [53]:
expand_email_df.tail()

,Date,From,To,Subject
11005,"Wed, 6 Sep 2023 01:23:34 +0000",Cynthia del Rosario,Lucylin03,Tapia/GHC PPT/ Alaska Mileage Plan sign-up
11006,"Wed, 6 Sep 2023 01:23:34 +0000",Cynthia del Rosario,Linh Pham,Tapia/GHC PPT/ Alaska Mileage Plan sign-up
11007,"Wed, 6 Sep 2023 01:23:34 +0000",Cynthia del Rosario,Nassim Parvin,Tapia/GHC PPT/ Alaska Mileage Plan sign-up
11008,"Wed, 6 Sep 2023 01:23:34 +0000",Cynthia del Rosario,Menita Agarwal,Tapia/GHC PPT/ Alaska Mileage Plan sign-up
11009,"Wed, 6 Sep 2023 01:23:34 +0000",Cynthia del Rosario,Isha Doshi,Tapia/GHC PPT/ Alaska Mileage Plan sign-up


# Prepare for edges

Calculate frenquncy for communication from the same sender to the same recipient

In [54]:
df1 = expand_email_df.iloc[:, 1:3]

In [55]:
df1['frequency'] = df1.groupby(['From', 'To'])['From'].transform('count')

In [56]:
df1.head()

,From,To,frequency
0,iSchool Career Services,me,258
1,Handshake,me,298
2,MSIM Academic Advising,me,16
3,Indeed,me,846
4,Ford Careers,me,4


# Prepare for nodes

In [57]:
# Get unique values from "From" and "To" columns
unique_from = df1['From'].unique()
unique_to = df1['To'].unique()

# Concatenate unique values into a new DataFrame
df2 = pd.concat([pd.DataFrame(unique_from, columns=['Unique']),
                              pd.DataFrame(unique_to, columns=['Unique'])], ignore_index=True)

# Remove duplicate values if any
df2.drop_duplicates(inplace=True)
df2.tail()

,Unique
1208,Rongrong Le
1210,gigi gonzales
1213,eng223
1214,iSchool MSIM Online
1216,IMAMuse


# Download Data

For Excel format

In [ ]:
# Create a Pandas Excel writer using ExcelWriter
with pd.ExcelWriter('output.xlsx') as writer:
    # Write each DataFrame to a separate sheet
    df2.to_excel(writer, sheet_name='nodes', index=False)  # Writing df2 to Sheet1 without index
    df1.to_excel(writer, sheet_name='edges', index=False)  # Writing df1 to Sheet2 without index

from google.colab import files

# Assuming you have already saved the Excel file
files.download('output.xlsx')

For JSON format

In [58]:
def dataframe_to_json_edges(df_edges, df_nodes):
    edges = df_edges.to_dict(orient='records')
    nodes = df_nodes.to_dict(orient='records')
    return {'nodes': nodes, 'edges': edges}

# Convert DataFrames to JSON
json_data = dataframe_to_json_edges(df1, df2)

# Save JSON data to a file
with open('combined_data.json', 'w') as f:
    json.dump(json_data, f)

# Download the file
from google.colab import files
files.download('combined_data.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Below are drafts to check all objects from original data

In [ ]:
all_keys = set()

# Iterate through each message in the mbox file
for message in mbox:
    msg = email.message_from_bytes(message.as_bytes())
    all_keys.update(msg.keys())

# Print all unique headers (keys)
print("All unique headers found in emails:")
for key in sorted(all_keys):
    print(key)

All unique headers found in emails:
ARC-Authentication-Results
ARC-Message-Signature
ARC-Seal
Accept-Language
Authentication-Results
Authentication-Results-Original
Auto-Submitted
CC
Cc
Charset
Content-Class
Content-ID
Content-Id
Content-Language
Content-Transfer-Encoding
Content-Type
Content-transfer-encoding
Content-type
DKIM-Filter
DKIM-Signature
Date
Deferred-Delivery
Delivered-To
DomainKey-Signature
Errors-To
FROM
Feedback-ID
Feedback-Id
From
Importance
In-Reply-To
IronPort-Data
IronPort-HdrOrdr
List-Archive
List-Help
List-ID
List-Id
List-Post
List-Subscribe
List-Unsubscribe
List-Unsubscribe-Post
MIME-Version
MIME-version
Message-ID
Message-Id
Message-id
Mime-Version
Origin-messageId
Original-Envelope-Id
Precedence
Priority
RPCampaign
Received
Received-SPF
Recipient-Id
References
Reply-To
Reply-to
Return-Path
Return-Receipt-To
Sender
Sender-IP
SenderEmail
SenderName
Sensitivity
Site-Id
Subject
TM-MAIL-JID
TO
Thread-Index
Thread-Topic
To
User-Agent
X-250ok-CID
X-ASG-Debug-ID
X-ASG-